In [20]:
# Dependências
import requests
import zipfile
import io
import pandas as pd
import unicodedata
import re

In [21]:
pd.options.display.max_columns = None

#### Dados Votação

In [22]:
# Download do arquivo ZIP em memória
response = requests.get("https://cdn.tse.jus.br/estatistica/sead/odsele/votacao_secao/votacao_secao_2022_RN.zip" )
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_file = [file for file in z.namelist() if file.endswith(".csv")][0]
        with z.open(csv_file) as f:
            df_votos = pd.read_csv(f, encoding="ISO-8859-1", sep=";")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {response.status_code}")

In [23]:
df_votos.drop(df_votos.loc[df_votos['DS_CARGO']!='DEPUTADO FEDERAL'].index, inplace=True)
df_votos.head(1)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
9,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,16217,AREIA BRANCA,32,24,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,3,1155,200001608805,ESCOLA MUNICIPAL COMERCIAL PROFESSORA GERALDA ...,"RUA JORGE CAMINHA, 118"


#### Dados Censitários - Por Distritos

In [24]:
'''
DPOs = Domicílios Particulares Ocupados
'''

df_distritos = pd.read_csv('Bases/Distritos.csv').rename(columns={'v0001': 'Pessoas',
                                                                  'v0002': 'Domicílios',
                                                                  'v0003': 'Domicílios Particulares',
                                                                  'v0004': 'Domicílios Coletivos',
                                                                  'v0005': 'Média de moradores em DPOs',
                                                                  'v0006': 'Percentual de DPOs Imputados',
                                                                  'v0007': 'Domicílios Particulares Ocupados'
                                                                  })
df_distritos.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,4750,4747,3,2.768747,0.996329,3814
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,24138,24103,35,2.932901,1.890418,19255
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,4339,4334,5,3.211385,0.177883,3373
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,1199,1199,0,2.972755,0.201816,991
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,6043,6040,3,2.811687,0.392319,4843


In [25]:
df_renda = pd.read_csv('Bases/Rendimento.csv').rename(columns={'Total': 'Total Indivíduos'})
df_renda.head()

,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,Total,2675748,288918,717278,323928,175559,60684,22246,7715,1078875,24
1,Acari,9537,1024,3101,1001,478,115,31,7,3779,2400109
2,Açu,44769,5554,13101,4715,2331,625,167,53,18208,2400208
3,Afonso Bezerra,8927,1789,2442,656,211,24,11,3,3791,2400307
4,Água Nova,2533,333,662,160,63,6,0,0,1309,2400406


In [26]:
df_censitario = pd.merge(df_distritos, df_renda, left_on='NM_DIST', right_on='Distrito', how='inner')
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,Acari,Acari,28,10597,4750,4747,3,2.768747,0.996329,3814,Acari,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,Açu,Açu,104,56496,24138,24103,35,2.932901,1.890418,19255,Açu,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,Afonso Bezerra,Afonso Bezerra,25,10839,4339,4334,5,3.211385,0.177883,3373,Afonso Bezerra,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,Água Nova,Água Nova,4,2946,1199,1199,0,2.972755,0.201816,991,Água Nova,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,Alexandria,Alexandria,22,13640,6043,6040,3,2.811687,0.392319,4843,Alexandria,11487,2191,3412,807,363,78,16,6,4614,2400505


In [27]:
'''
Normalizando os nomes dos distritos para eles baterem com df_votos
'''

def normalize_text(value):
    if isinstance(value, str):
        # Removendo acentos e caracteres especiais
        value = ''.join(
            char for char in unicodedata.normalize('NFD', value)
            if unicodedata.category(char) != 'Mn'
        )
        # Convertendo para letra maiúscula
        value = value.upper()
        # Removendo caracteres não-alfanuméricos exceto espaços
        value = re.sub(r'[^A-Z0-9 ]', '', value)
    return value

# Aplicando a função nas colunas
df_censitario['NM_MUN'] = df_censitario['NM_MUN'].apply(normalize_text)
df_censitario['NM_DIST'] = df_censitario['NM_DIST'].apply(normalize_text)
df_censitario.head()

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,2,Nordeste,24,Rio Grande do Norte,2400109,240010905,ACARI,ACARI,28,10597,4750,4747,3,2.768747,0.996329,3814,Acari,9537,1024,3101,1001,478,115,31,7,3779,2400109
1,2,Nordeste,24,Rio Grande do Norte,2400208,240020805,ACU,ACU,104,56496,24138,24103,35,2.932901,1.890418,19255,Açu,44769,5554,13101,4715,2331,625,167,53,18208,2400208
2,2,Nordeste,24,Rio Grande do Norte,2400307,240030705,AFONSO BEZERRA,AFONSO BEZERRA,25,10839,4339,4334,5,3.211385,0.177883,3373,Afonso Bezerra,8927,1789,2442,656,211,24,11,3,3791,2400307
3,2,Nordeste,24,Rio Grande do Norte,2400406,240040605,AGUA NOVA,AGUA NOVA,4,2946,1199,1199,0,2.972755,0.201816,991,Água Nova,2533,333,662,160,63,6,0,0,1309,2400406
4,2,Nordeste,24,Rio Grande do Norte,2400505,240050505,ALEXANDRIA,ALEXANDRIA,22,13640,6043,6040,3,2.811687,0.392319,4843,Alexandria,11487,2191,3412,807,363,78,16,6,4614,2400505


#### Dados Unificados

In [28]:
df_unificado = pd.merge(df_votos, df_censitario, left_on='NM_MUNICIPIO', right_on='NM_DIST', how='inner')
df_unificado.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,CD_DIST,NM_MUN,NM_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados,Distrito,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
0,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,16217,AREIA BRANCA,32,24,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,3,1155,200001608805,ESCOLA MUNICIPAL COMERCIAL PROFESSORA GERALDA ...,"RUA JORGE CAMINHA, 118",2,Nordeste,24,Rio Grande do Norte,2401107,240110705,AREIA BRANCA,AREIA BRANCA,42,24093,11691,11680,11,2.942442,2.835146,8183,Areia Branca,21514,2118,5343,2759,1575,376,55,8,9276,2401107
1,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,18279,SANTANA DO MATOS,18,119,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,85,1147,200001608805,ESCOLA MUNICIPAL LUIZ LIBERALINO DE CARVALHO,"POVOADO BOM JESUS, S/N",2,Nordeste,24,Rio Grande do Norte,2411403,241140305,SANTANA DO MATOS,SANTANA DO MATOS,17,8688,4192,4190,2,2.769525,0.127510,3137,Santana do Matos,11873,2175,3384,598,230,46,9,5,5426,2411403
2,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,17612,NATAL,4,308,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,5,1546,200001608805,ESCOLA ESTADUAL ANTONIO PINTO DE MEDEIROS,"RUA RIO PARANAPANEMA, S/N - CONJUNTO CIDADE SA...",2,Nordeste,24,Rio Grande do Norte,2408102,240810205,NATAL,NATAL,1116,751300,337029,336655,374,2.772461,3.632069,270507,Natal,695027,32863,165649,115979,73350,32817,14692,5692,253853,2408102
3,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,17493,MAXARANGUAPE,64,39,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,43,1066,200001608805,ESCOLA MUNICIPAL PROFESSOR ESMERINO GOMES DE S...,"RUA DA MACAÍBA, SN - PRAIA DE CARAUBAS",2,Nordeste,24,Rio Grande do Norte,2407500,240750005,MAXARANGUAPE,MAXARANGUAPE,23,10255,5576,5564,12,3.181790,0.590429,3218,Maxaranguape,8365,1154,2473,761,216,55,18,6,3682,2407500
4,01/11/2022,16:05:25,2022,2,ELEIÇÃO ORDINÁRIA,1,546,ELEIÇÕES GERAIS ESTADUAIS 2022,02/10/2022,E,RN,RN,RIO GRANDE DO NORTE,17876,PAU DOS FERROS,40,89,6,DEPUTADO FEDERAL,4444,LUIZ BENES LEOCÁDIO DE ARAÚJO,1,1139,200001608805,ESCOLA ESTADUAL TEOFILO REGO,"RUA DA INDEPENDENCIA, 60",2,Nordeste,24,Rio Grande do Norte,2409407,240940705,PAU DOS FERROS,PAU DOS FERROS,41,30479,13051,13041,10,2.805045,1.938416,10782,Pau dos Ferros,23678,2364,7629,2585,1603,460,128,39,8869,2409407


#### Análise Exploratória

##### Dados Faltantes

Como a base com informações de renda é de 2010 e a base com informações censitárias é de 2022, há divergências na nomeclatura de alguns municípios e distritos. O jeito que escolhemos tratar essas divergências foi dropando as linhas referentes à municípios que só aparecem em uma das bases para evitar possíveis conflitos. Por conta disso esses dados vão estar faltando nas análises realizadas neste trabalho.

In [29]:
# Procurando dados faltantes no df unificado

missing_data_summary = df_unificado.isnull().sum()
missing_data_summary

DT_GERACAO                             0
HH_GERACAO                             0
ANO_ELEICAO                            0
CD_TIPO_ELEICAO                        0
NM_TIPO_ELEICAO                        0
NR_TURNO                               0
CD_ELEICAO                             0
DS_ELEICAO                             0
DT_ELEICAO                             0
TP_ABRANGENCIA                         0
SG_UF                                  0
SG_UE                                  0
NM_UE                                  0
CD_MUNICIPIO                           0
NM_MUNICIPIO                           0
NR_ZONA                                0
NR_SECAO                               0
CD_CARGO                               0
DS_CARGO                               0
NR_VOTAVEL                             0
NM_VOTAVEL                             0
QT_VOTOS                               0
NR_LOCAL_VOTACAO                       0
SQ_CANDIDATO                           0
NM_LOCAL_VOTACAO

Existem 4843 dados faltantes na coluna "Percentual de DPOs Imputados", mas como provavelmente não precisaremos utilizar esses dados nas análises futuras, escolhemos manter as linhas com esses dados faltantes ao invés de removê-los da base de dados. Caso seja necessários realizar análises que utilizam estes dados no futuro poderemos reavaliar esta escolha.

##### Limitações dos dados

A maior limitação dos dados utilizados é o fato dos dados eleitorais e dos distritos serem de 2022, enquanto as informações sobre renda são de 2010. Isso pode levar a dificuldades futuras nas análises, uma vez que as informações podem estar desatualizadas ou até mesmo causar incompatibilidades entre as bases.

##### Necessidade de correção de problemas de padronização nas colunas

Os nomes dos municípios e distritos na base do IBGE estavam escritos com acentos e apenas a primeira letra maiúscula, enquanto na base de votação os nomes estavam sem caracteres especiais e completamente capitalizados, então os nomes de distritos na base censitária foi convertida para o mesmo formato da base de votação para que as duas bases pudessem ser unificadas

##### Verificação das faixas de variação dos valores das variáveis, estatísticas básicas sobre as variáveis (média, desvio padrão, etc.), valores possíveis de atributos categóricos

In [30]:
# Estatísticas básicas para variáveis numéricas, arredondadas
numeric_stats = df_unificado.describe().round(2)

# Estatísticas adicionais: média, desvio padrão e faixa, arredondadas
additional_stats = df_unificado.select_dtypes(include=['number']).agg(['mean', 'std', 'min', 'max']).round(2)

# Valores únicos para variáveis categóricas
categorical_values = {col: df_unificado[col].unique() for col in df_unificado.select_dtypes(include=['object']).columns}

In [31]:
numeric_stats

,ANO_ELEICAO,CD_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,CD_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,NR_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,CD_REGIAO,CD_UF,CD_MUN,CD_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
count,207253.0,207253.0,207253.0,207253.0,207253.00,207253.00,207253.00,207253.0,207253.00,207253.00,207253.00,2.072530e+05,207253.0,207253.0,207253.0,2.072530e+05,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,202410.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.00,207253.0
mean,2022.0,2.0,1.0,546.0,17469.35,26.82,196.87,6.0,2574.98,6.19,1449.47,1.646088e+11,2.0,24.0,2407059.4,2.407059e+08,509.48,343215.63,154056.65,153894.48,162.17,2.86,2.72,123296.59,310265.64,15294.71,73819.30,51137.69,32722.68,14534.73,6354.82,2407.01,113936.05,2407059.4
std,0.0,0.0,0.0,0.0,655.33,23.69,169.15,0.0,2131.88,11.87,448.98,7.632819e+10,0.0,0.0,3404.2,3.404203e+05,512.89,346571.60,155402.86,155228.32,176.98,0.13,1.41,124983.19,323989.21,14759.43,77149.96,54582.97,34441.61,15482.88,7000.59,2743.41,117864.71,3404.2
min,2022.0,2.0,1.0,546.0,16004.00,1.00,1.00,6.0,10.00,1.00,1015.00,-3.000000e+00,2.0,24.0,2400109.0,2.400109e+08,4.00,1335.00,573.00,573.00,0.00,2.62,0.04,437.00,1756.00,231.00,458.00,129.00,44.00,5.00,0.00,0.00,527.00,2400109.0
25%,2022.0,2.0,1.0,546.0,17310.00,3.00,56.00,6.0,1100.00,1.00,1066.00,2.000016e+11,2.0,24.0,2403608.0,2.403608e+08,30.00,13577.00,5757.00,5752.00,6.00,2.77,1.46,4498.00,10858.00,1624.00,3100.00,917.00,363.00,75.00,15.00,5.00,4600.00,2403608.0
50%,2022.0,2.0,1.0,546.0,17612.00,20.00,128.00,6.0,2007.00,2.00,1236.00,2.000016e+11,2.0,24.0,2408102.0,2.408102e+08,327.00,243612.00,109395.00,109361.00,34.00,2.80,3.63,86922.00,172057.00,7574.00,37148.00,27655.00,22083.00,9519.00,2986.00,659.00,64393.00,2408102.0
75%,2022.0,2.0,1.0,546.0,17795.00,50.00,339.00,6.0,4422.00,6.00,1821.00,2.000016e+11,2.0,24.0,2408102.0,2.408102e+08,1116.00,751300.00,337029.00,336655.00,374.00,2.92,3.63,270507.00,695027.00,32863.00,165649.00,115979.00,73350.00,32817.00,14692.00,5692.00,253853.00,2408102.0
max,2022.0,2.0,1.0,546.0,18996.00,69.00,630.00,6.0,7799.00,335.00,2577.00,2.000017e+11,2.0,24.0,2415008.0,2.415008e+08,1116.00,751300.00,337029.00,336655.00,374.00,3.36,4.44,270507.00,695027.00,32863.00,165649.00,115979.00,73350.00,32817.00,14692.00,5692.00,253853.00,2415008.0


In [32]:
additional_stats

,ANO_ELEICAO,CD_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,CD_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,NR_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,SQ_CANDIDATO,CD_REGIAO,CD_UF,CD_MUN,CD_DIST,Total_setores,Pessoas,Domicílios,Domicílios Particulares,Domicílios Coletivos,Média de moradores em DPOs,Percentual de DPOs Imputados,Domicílios Particulares Ocupados,Total Indivíduos,Até 1/2,Mais de 1/2 a 1,Mais de 1 a 2,Mais de 2 a 5,Mais de 5 a 10,Mais de 10 a 20,Mais de 20,Sem rendimento,Código da Unidade Geográfica
mean,2022.0,2.0,1.0,546.0,17469.35,26.82,196.87,6.0,2574.98,6.19,1449.47,1.646088e+11,2.0,24.0,2407059.4,2.407059e+08,509.48,343215.63,154056.65,153894.48,162.17,2.86,2.72,123296.59,310265.64,15294.71,73819.30,51137.69,32722.68,14534.73,6354.82,2407.01,113936.05,2407059.4
std,0.0,0.0,0.0,0.0,655.33,23.69,169.15,0.0,2131.88,11.87,448.98,7.632819e+10,0.0,0.0,3404.2,3.404203e+05,512.89,346571.60,155402.86,155228.32,176.98,0.13,1.41,124983.19,323989.21,14759.43,77149.96,54582.97,34441.61,15482.88,7000.59,2743.41,117864.71,3404.2
min,2022.0,2.0,1.0,546.0,16004.00,1.00,1.00,6.0,10.00,1.00,1015.00,-3.000000e+00,2.0,24.0,2400109.0,2.400109e+08,4.00,1335.00,573.00,573.00,0.00,2.62,0.04,437.00,1756.00,231.00,458.00,129.00,44.00,5.00,0.00,0.00,527.00,2400109.0
max,2022.0,2.0,1.0,546.0,18996.00,69.00,630.00,6.0,7799.00,335.00,2577.00,2.000017e+11,2.0,24.0,2415008.0,2.415008e+08,1116.00,751300.00,337029.00,336655.00,374.00,3.36,4.44,270507.00,695027.00,32863.00,165649.00,115979.00,73350.00,32817.00,14692.00,5692.00,253853.00,2415008.0


In [33]:
# Ordenar valores categóricos pelo número de itens únicos
sorted_categorical_values = {col: values for col, values in sorted(
    categorical_values.items(), key=lambda item: len(item[1]), reverse=False
)}

# Exibir os valores categóricos ordenados
print("\nValores únicos para variáveis categóricas (ordenados por número de itens):")
for col, values in sorted_categorical_values.items():
    print(f"{col} ({len(values)} valores únicos): {values}")
    print()


Valores únicos para variáveis categóricas (ordenados por número de itens):
DT_GERACAO (1 valores únicos): ['01/11/2022']

HH_GERACAO (1 valores únicos): ['16:05:25']

NM_TIPO_ELEICAO (1 valores únicos): ['ELEIÇÃO ORDINÁRIA']

DS_ELEICAO (1 valores únicos): ['ELEIÇÕES GERAIS ESTADUAIS 2022']

DT_ELEICAO (1 valores únicos): ['02/10/2022']

TP_ABRANGENCIA (1 valores únicos): ['E']

SG_UF (1 valores únicos): ['RN']

SG_UE (1 valores únicos): ['RN']

NM_UE (1 valores únicos): ['RIO GRANDE DO NORTE']

DS_CARGO (1 valores únicos): ['DEPUTADO FEDERAL']

NM_REGIAO (1 valores únicos): ['Nordeste']

NM_UF (1 valores únicos): ['Rio Grande do Norte']

NM_MUNICIPIO (96 valores únicos): ['AREIA BRANCA' 'SANTANA DO MATOS' 'NATAL' 'MAXARANGUAPE' 'PAU DOS FERROS'
 'ACARI' 'ALTO DO RODRIGUES' 'MONTANHAS' 'LAJES PINTADAS'
 'LAGOA DE PEDRAS' 'NOVA CRUZ' 'RUY BARBOSA' 'MACAU' 'SANTA CRUZ'
 'EXTREMOZ' 'PARNAMIRIM' 'PATU' 'PEDRO AVELINO' 'TOUROS'
 'FERNANDO PEDROZA' 'TIBAU DO SUL' 'LAGOA DE VELHOS' 'LAJES'
 

#### Banco de dados

In [34]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
pip install pandas sqlalchemy psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
from conexao import criar_conexao
engine = criar_conexao()
tabela='dados_eleitorais'
df_unificado.to_sql(tabela, con=engine, if_exists='replace', index=False)

print(f"Tabela '{tabela}' criada ou atualizada com sucesso no banco de dados!")

Tabela 'dados_eleitorais' criada ou atualizada com sucesso no banco de dados!
